In [1]:
import pandas as pd 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

In [2]:
# 접속 주소
url = "https://comp.wisereport.co.kr/company/c1010001.aspx?cmp_cd="
# 종목의 코드 (하이닉스 종목코드)
input_code = "000660"

### 연습 
- webdriver의 Chrome을 이용하여 url + input_code 로 요청 
- driver의 html 문서를 불러온다. 
- 불러온 html을 BeautifulSoup을 이용하여 데이터 파싱 
- div 태그 중 class의 이름이 "cmp_comment"와 같은 태그를 선택 
    - li 태그들을 모두 찾아서 텍스트 추출하여 새로운 리스트를 생성(cmp_info)
- div 태그 중 class의 이름이 "cmp_comment_etc"와 같은 태그를 선택 
    - li 태그들을 모두 찾아서 텍스트 추출하여 새로운 리스트를 생성(cmp_etc)
- DataFrame 생성하여 cmp_info, cmp_etc 을 대입 

In [11]:
driver = webdriver.Chrome()

In [12]:
# url + input_code를 이용해서 요청 
driver.get(url+input_code)

In [13]:
soup = bs(
    driver.page_source, 'html.parser'
)

In [14]:
driver.quit()

In [15]:
# div 태그중 class가 'cmp_comment'인 태그를 선택 
div_data = soup.find(
    'div', attrs={
        'class' : 'cmp_comment'
    }
)
div_data

<div class="cmp_comment">
<ul class="dot_cmp">
<li class="dot_cmp" data-cd="000660">동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연구개발법인 및 여러 해외 판매법인을 운영하는 글로벌 반도체 기업임.</li>
<li class="dot_cmp" data-cd="000660">메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주력 생산하고 있음.</li>
<li class="dot_cmp" data-cd="000660">생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔루션 개발에 주력하고 있음.</li>
</ul>
</div>

In [17]:
# div_data에서 li 태그들을 모두 찾는다. 
li_list = div_data.find_all('li')

In [18]:
cmp_info = []
for li_data in li_list:
    # li_data는 Tag data 대입 
    cmp_info.append(li_data.get_text())
cmp_info

['동사는 1949년 설립되어 경기도 이천시에 본사를 두고 4개의 생산기지와 3개의 연구개발법인 및 여러 해외 판매법인을 운영하는 글로벌 반도체 기업임.',
 '메모리 반도체와 Foundry 사업을 영위하며, DRAM과 NAND Flash를 주력 생산하고 있음.',
 '생성형 AI 시장의 신기술 등장과 시장 영향력 확대에 따라 고성능/고용량 메모리 솔루션 개발에 주력하고 있음.']